# Markov chain-based approach for the music generation

In [1]:
import numpy as np
import pandas as pd
from collections import Counter
np.random.seed(42)
from music21 import*

s = converter.parse(r'C:\Users\maxwi\JazzCorpus\ChetBakerLetsGetLost.mid')
pitches = s.recurse().notes
solodata = []
#prints all notes in solo
for n in s.flatten().getElementsByClass(note.Note):
    solodata.append(n.pitch.name)


# read file
data = pd.read_csv('data/Liverpool_band_chord_sequence.csv')

## Generate bigrams

In [2]:
n = 2
notes = data['notes'].values
ngrams = zip(*[notes[i:] for i in range(n)])
bigrams = [" ".join(ngram) for ngram in ngrams]

bigrams[:5]

['B B', 'B A', 'A B', 'B C', 'C D']

## Predict next state with Markov chain

In [3]:
def predict_next_state(chord:str, data:list=bigrams):
    """Predict next chord based on current state."""
    # create list of bigrams which stats with current chord
    bigrams_with_current_chord = [bigram for bigram in bigrams if bigram.split(' ')[0]==chord]
    # count appearance of each bigram
    count_appearance = dict(Counter(bigrams_with_current_chord))
    # convert apperance into probabilities
    for ngram in count_appearance.keys():
        count_appearance[ngram] = count_appearance[ngram]/len(bigrams_with_current_chord)
    # create list of possible options for the next chord
    options = [key.split(' ')[1] for key in count_appearance.keys()]
    # create  list of probability distribution
    probabilities = list(count_appearance.values())
    # return random prediction
    return np.random.choice(options, p=probabilities)

In [4]:
# example
predict_next_state('F')

'A'

## Generate sequence

In [5]:
def generate_sequence(chord:str=None, data:list=bigrams, length:int=30):
    """Generate sequence of defined length."""
    # create list to store future chords
    chords = []
    for n in range(length):
        # append next chord for the list
        chords.append(predict_next_state(chord, bigrams))
        # use last chord in sequence to predict next chord
        chord = chords[-1]
    return chords

In [6]:
generate_sequence('C')

['G',
 'E',
 'D',
 'B',
 'B',
 'B',
 'G',
 'E',
 'D',
 'B',
 'D',
 'F',
 'A',
 'B',
 'B',
 'A',
 'E',
 'E',
 'G',
 'E',
 'G',
 'G',
 'G',
 'F',
 'E',
 'G',
 'E',
 'D',
 'B',
 'C']